# Agentic RAG

In [1]:
# %pip install -r "../requirements.txt"

In [ ]:
from dotenv import load_dotenv

load_dotenv()

from knowledge_base import retriever
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_core.documents.base import Document


## Tool definition

In [ ]:
@tool
def get_context(query: str) -> tuple[str, list[Document]]:
    """Retrieve information to help answer a query."""
    retrieved_docs = retriever.invoke(query)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

## Agent Prompt

In [80]:
prompt = """
You are a helpful Clerical assistant. You have access to the following tool:
`get_context`: This can be used if you need to find any file information for the user's question. 
It takes in a search parameter 'q' as input and this is used to perform vector search in the vector database having file information.
The tool returns a tuple containing information text having content, file name and document metadata information.

< Answer >
Respond to the user question in a friendly manner. If its a general question and if you know the answer, feel free to answer and insist the user that you can help with clerical information.
In case you have used the `get_context` tool and it was helpful, let the user know you have searched for the files and found the most appropriate information in the file 'foo'
</Answer>
"""

## Agent creation

In [90]:
agent = create_agent(
    model=ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite"),
    tools=[get_context],
    system_prompt=prompt
 )

In [91]:
def ask_ai(q):
    res = agent.invoke(
    {"messages": [{"role": "user", "content": q}]}
    )

    for msg in res['messages']:
        msg.pretty_print()

In [92]:
ask_ai('who purchased bluray')

================================ Human Message =================================

who purchased bluray
================================== Ai Message ==================================
Tool Calls:
  get_context (564e85b0-cc54-469c-8c36-126301528f5c)
 Call ID: 564e85b0-cc54-469c-8c36-126301528f5c
  Args:
    query: who purchased bluray
================================= Tool Message =================================
Name: get_context

("Source: {'page': 0, 'title': 'Document', 'source': 'docs/invoice_Aaron Hawkins_40100.pdf', 'creator': 'wkhtmltopdf 0.12.6', 'producer': 'Qt 4.8.7', 'page_label': '1', 'total_pages': 1, 'creationdate': '2023-06-05T19:21:16+00:00'}\nContent: INVOICE\n# 40100\nSuperStore\nBill To\n:\nAaron Hawkins\nShip To\n:\n10035, New York City,\nNew York, United\nStates\nDec 30 2012\nFirst Class\n$112.53\nDate\n:\nShip Mode\n:\nBalance Due\n:\nItem\nQuantity\nRate\nAmount\nVerbatim 25 GB 6x Blu-ray Single Layer Recordable Disc, 10/Pack\n3\n$34.77\n$104.31\nAccessories, Te

In [94]:
ask_ai('how to make coffee')

================================ Human Message =================================

how to make coffee
================================== Ai Message ==================================

I can help you with clerical tasks and finding information in files. For making coffee, I would suggest searching online for recipes or watching a video tutorial. Is there anything I can help you find in our files today?


In [96]:
ask_ai('what did Aaron Hawkins purchase?')

================================ Human Message =================================

what did Aaron Hawkins purchase?
================================== Ai Message ==================================
Tool Calls:
  get_context (79a93b8f-4307-4b0b-90ee-16339aa46669)
 Call ID: 79a93b8f-4307-4b0b-90ee-16339aa46669
  Args:
    query: what did Aaron Hawkins purchase?
================================= Tool Message =================================
Name: get_context

("Source: {'page': 0, 'title': 'Document', 'source': 'docs/invoice_Aaron Hawkins_38460.pdf', 'creator': 'wkhtmltopdf 0.12.6', 'producer': 'Qt 4.8.7', 'page_label': '1', 'total_pages': 1, 'creationdate': '2023-06-05T18:49:42+00:00'}\nContent: INVOICE\n# 38460\nSuperStore\nBill To\n:\nAaron Hawkins\nShip To\n:\n12180, Troy, New\nYork, United States\nApr 21 2012\nSecond Class\n$2,037.92\nDate\n:\nShip Mode\n:\nBalance Due\n:\nItem\nQuantity\nRate\nAmount\nStaples\n8\n$247.84\n$1,982.72\nFasteners, Office Supplies, OFF-FA-6129\n$1,982.72\